# 8. NAIVE BAYES


Contenido:
1. Introducción y teoría de decisión bayesiana
2. Probabilidad condicional y la suposición "naive"
3. Clasificación con Naive Bayes (Gaussian, Multinomial, Bernoulli)
4. Ejemplos ilustrativos (Iris, texto reducido con 20newsgroups)
5. Regresión con una aproximación Naive (discretización)
6. Implementación desde cero (Gaussian Naive Bayes, brute-force)
7. Selección de modelo, validación y recomendaciones

## 1. Introducción y teoría de decisión bayesiana

La **teoría de la decisión bayesiana** proporciona un marco probabilístico para la clasificación: se asigna a una observación $x$ la clase $k$ que maximiza la probabilidad a posteriori $P(Y=k\mid X=x)$, considerando una pérdida adecuada.

La regla bayesiana de mínima probabilidad de error (0-1 loss) es:

\begin{equation}
\hat{y}(x) = \arg\max_{k} P(Y=k \mid X=x)
\end{equation}

Por Bayes:
\begin{equation}
P(Y=k \mid X=x) = \frac{P(X=x \mid Y=k) P(Y=k)}{P(X=x)}.
\end{equation}

Como $P(X=x)$ es común a todas las clases, la decisión se reduce a comparar $P(X=x\mid Y=k)P(Y=k)$ entre clases.

## 2. Probabilidad condicional y la suposición "naive"

El término *Naive* proviene de la suposición de **independencia condicional** entre las características dado $Y$:

\begin{equation}
P(X=x \mid Y=k) = \prod_{j=1}^d P(X_j = x_j \mid Y=k).
\end{equation}

Bajo esta suposición, el cálculo de la verosimilitud se simplifica enormemente y la regla de decisión se vuelve:
\begin{equation}
\hat{y}(x) = \arg\max_k P(Y=k) \prod_{j=1}^d P(X_j = x_j \mid Y=k).
\end{equation}

Para evitar underflow numérico se usa la versión logarítmica (log-sum):
\begin{equation}
\hat{y}(x) = \arg\max_k \left( \log P(Y=k) + \sum_{j=1}^d \log P(X_j = x_j \mid Y=k) \right).
\end{equation}

Dependiendo de la naturaleza de las variables $X_j$ elegimos distintos modelos para $P(X_j \mid Y=k)$: Gaussiano para variables continuas, multinomial o Bernoulli para conteos/binarios, etc.

## 3. Clasificación con Naive Bayes — Formulaciones comunes

- **Gaussian Naive Bayes (GNB):** assume $X_j\mid Y=k \sim \mathcal{N}(\mu_{jk}, \sigma_{jk}^2)$. Entonces
\begin{equation}
P(X=x \mid Y=k) = \prod_{j=1}^d \frac{1}{\sqrt{2\pi\sigma_{jk}^2}} \exp\left(-\frac{(x_j-\mu_{jk})^2}{2\sigma_{jk}^2}\right).
\end{equation}

- **Multinomial Naive Bayes:** usado para conteos de palabras, modela la probabilidad de la secuencia de conteos dados parámetros de tasa por clase.

- **Bernoulli Naive Bayes:** variables binarias (presencia/ausencia de términos).

En sklearn: `GaussianNB`, `MultinomialNB`, `BernoulliNB`.

In [1]:
# Ejemplo práctico: Gaussian Naive Bayes en Iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('Accuracy (GNB, Iris):', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=iris.target_names))

Accuracy (GNB, Iris): 0.9666666666666667
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        11
  versicolor       0.93      1.00      0.96        13
   virginica       1.00      0.83      0.91         6

    accuracy                           0.97        30
   macro avg       0.98      0.94      0.96        30
weighted avg       0.97      0.97      0.97        30



### Observaciones prácticas

- GNB estima para cada característica y cada clase la media $\mu_{jk}$ y varianza $\sigma_{jk}^2$ (método de momentos / MLE bajo normalidad).
- MultinomialNB requiere features no negativas (conteos o TF-IDF). BernoulliNB requiere binarias.
- Naive Bayes es extremadamente rápido y robusto en alta dimensión cuando la independencia aproximada es razonable (por ejemplo, texto).

## 4. Ejemplo ilustrativo con texto (MultinomialNB)

Usaremos `fetch_20newsgroups` pero restringiremos a dos categorías para mantenerlo ligero y claro: *sci.space* y *rec.sport.hockey* (clasificación binaria).

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

categories = ['sci.space', 'rec.sport.hockey']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers','footers','quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=('headers','footers','quotes'))

text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english', max_df=0.7)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
text_clf.fit(newsgroups_train.data, newsgroups_train.target)
predicted = text_clf.predict(newsgroups_test.data)
print('Accuracy (MultinomialNB, 20newsgroups subset):', accuracy_score(newsgroups_test.target, predicted))
print(confusion_matrix(newsgroups_test.target, predicted))

Accuracy (MultinomialNB, 20newsgroups subset): 0.9508196721311475
[[391   8]
 [ 31 363]]


## 5. Regresión con Naive Bayes — aproximación práctica (discretización)

Naive Bayes es nativamente un clasificador. Para ilustrar una adaptación para problemas de regresión podemos **discretizar** la variable continua $y$ en bins y usar un clasificador (p. ej. MultinomialNB) para predecir el bin; la predicción continua se obtiene usando el valor promedio (o mediana) del bin predicho.

Formalmente: definir partición $\{B_1,\dots,B_m\}$ del rango de $Y$. Aprender $P(Y\in B_r \mid X=x)$ y predecir
\begin{equation}
\hat{y}(x) = E[Y \mid X=x] \approx \sum_{r=1}^m m_r \; P(Y\in B_r \mid X=x),
\end{equation}

donde $m_r$ es el representante del bin $B_r$ (por ejemplo, su media). A continuación un ejemplo con `diabetes` discretizado.

In [3]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error

Xd, yd = load_diabetes(return_X_y=True)
Xtr_d, Xte_d, ytr_d, yte_d = train_test_split(Xd, yd, test_size=0.3, random_state=0)

# Discretizar y en m bins (ordinal)
m = 10
kbd = KBinsDiscretizer(n_bins=m, encode='ordinal', strategy='quantile')
ytr_bins = kbd.fit_transform(ytr_d.reshape(-1,1)).ravel().astype(int)
yte_bins = kbd.transform(yte_d.reshape(-1,1)).ravel().astype(int)

# Entrenar GaussianNB sobre características continuas pero etiquetas discretas
gnb_reg = GaussianNB()
gnb_reg.fit(Xtr_d, ytr_bins)
pred_bins = gnb_reg.predict(Xte_d)

# Convertir bin predicho a valor real (media del bin en entrenamiento)
bin_means = {b: ytr_d[ytr_bins==b].mean() for b in np.unique(ytr_bins)}
y_pred_cont = np.array([bin_means[b] for b in pred_bins])
print('Approx regression MSE via discretized NB:', mean_squared_error(yte_d, y_pred_cont))

Approx regression MSE via discretized NB: 4684.779436862775


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


## 6. Implementación de Gaussian Naive Bayes desde cero (brute-force)

Implementamos estimadores MLE para $\mu_{jk}, \sigma_{jk}^2$ y la regla de decisión logarítmica para evitar underflow.

Esta implementación sirve para fines pedagógicos y para comprobar la equivalencia con `sklearn.naive_bayes.GaussianNB` en pequeños datasets.

In [4]:
class GaussianNBBrute:
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing
    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.classes_, counts = np.unique(y, return_counts=True)
        self.class_prior_ = {c: cnt/len(y) for c, cnt in zip(self.classes_, counts)}
        self.theta_ = {}  # mean per class and feature
        self.sigma_ = {}  # var per class and feature
        for c in self.classes_:
            Xc = X[y==c]
            self.theta_[c] = Xc.mean(axis=0)
            # var MLE (biased) or unbiased? sklearn uses var with N, we use var with ddof=0
            self.sigma_[c] = Xc.var(axis=0) + self.var_smoothing
        return self
    def _log_likelihood(self, x):
        # returns dict class -> log p(x|class) + log prior
        res = {}
        for c in self.classes_:
            mu = self.theta_[c]
            var = self.sigma_[c]
            # Gaussian log-likelihood (sum over features)
            ll = -0.5 * np.sum(np.log(2 * np.pi * var)) - 0.5 * np.sum(((x - mu)**2) / var)
            res[c] = ll + np.log(self.class_prior_[c])
        return res
    def predict(self, Xq):
        Xq = np.atleast_2d(Xq)
        preds = []
        for x in Xq:
            ll = self._log_likelihood(x)
            preds.append(max(ll.items(), key=lambda kv: kv[1])[0])
        return np.array(preds)

# Comprobación con Iris (usar todo el dataset)
clf_custom = GaussianNBBrute().fit(X_train, y_train)
pred_custom = clf_custom.predict(X_test)
print('Custom GNB accuracy on Iris test (brute):', accuracy_score(y_test, pred_custom))

# Comparación con sklearn
pred_sklearn = gnb.predict(X_test)
print('Sklearn GNB accuracy on Iris test:', accuracy_score(y_test, pred_sklearn))


Custom GNB accuracy on Iris test (brute): 0.9666666666666667
Sklearn GNB accuracy on Iris test: 0.9666666666666667


## 7. Selección de variantes y validación

- Use **GaussianNB** para features continuas que aproximen normalidad por clase.  
- Use **MultinomialNB** para conteos / text data (previo `CountVectorizer` o `TfidfTransformer`).  
- Use **BernoulliNB** para features binarias (p.ej. presencia/ausencia).  
- Validar con *cross-validation* y calibrar prior si la clase está desbalanceada (parámetro `class_prior` en implementaciones).  
- En problemas reales, la independencia condicional raramente es exacta; Naive Bayes suele funcionar bien por su parsimonia.

### Recomendaciones finales
- Reportar accuracy, precision/recall/F1 según el problema.  
- Para texto: limpiar, eliminar stopwords, y considerar n-gramas con regularización al contar features.  
- Para regresión via discretización: evaluar la granularidad de bins y la pérdida de información.

### Referencias
- Duda, R. O., Hart, P. E., & Stork, D. G. (2001). *Pattern Classification*.
- Bishop, C. M. (2006). *Pattern Recognition and Machine Learning*. (Capítulo sobre métodos generativos)
- McCallum, A., & Nigam, K. (1998). A comparison of event models for naive bayes text classification.